In [1]:
import pandas as pd

from src.data.models import Dataset

In [2]:
legislatures = Dataset(
    name="legislatures",
    awde_endpoint="parliament-periods",
    awde_params={
        "label": "Bundestag 2021 - 2025",
        "type": "legislature",
    },
)
legislatures.fetch()

In [3]:
legislatures.data

,id,entity_type,label,api_url,abgeordnetenwatch_url,parliament,previous_period,type,election_date,start_date_period,end_date_period
0,132,parliament_period,Bundestag 2021 - 2025,https://www.abgeordnetenwatch.de/api/v2/parlia...,https://www.abgeordnetenwatch.de/bundestag,"{'id': 5, 'entity_type': 'parliament', 'label'...","{'id': 128, 'entity_type': 'parliament_period'...",legislature,None,2021-09-29,2025-10-26


In [4]:
# Abstimmungen im aktuellen Bundestag:
polls = Dataset(
    name="polls",
    awde_endpoint="polls",
    awde_params={"field_legislature[entity.id]": 132}
)

def transform_polls(data: pd.DataFrame) -> pd.DataFrame:
    df = data.copy()
    df.rename(
        {
            "field_accepted": "accepted",
            "field_poll_date": "date"
        },
        axis=1,
        inplace=True
    )
    df["fid_legislatur"] = df.field_legislature.apply(lambda x: x["id"])
    df["fid_topic"] = df.field_topics.apply(lambda x: ",".join( [str(i["id"]) for i in x] ) if x is not None else None)
    df = df[["id", "fid_legislatur", "fid_topic", "label", "date", "accepted"]]

    return df

polls.transform_data = transform_polls

In [5]:
polls.data

,id,fid_legislatur,fid_topic,label,date,accepted
0,5593,132,11,Zweite Beratung zur Aufhebung des Lieferketten...,2024-06-13,False
1,5592,132,16,Ablehnung eines Antrags zum Verbot des Vereins...,2024-06-13,True
2,5591,132,"23,25",Konsequenzen aus der tödlichen Messerattacke i...,2024-06-13,False
3,5570,132,"23,25",Ablehnung eines Antrags zur Bekämpfung der Gef...,2024-06-06,True
4,5569,132,"20,4,9",Verbesserung des Klimaschutzes beim Immissions...,2024-06-06,True
...,...,...,...,...,...,...
138,4446,132,"22,42",Zweiter Nachtrag zum Bundeshaushalt 2021,2022-01-27,True
139,4388,132,"2,28",DIE LINKE-Antrag: Prämie für Beschäftigte im G...,2021-12-10,False
140,4386,132,28,Impfpflicht für bestimmte Berufsgruppen und we...,2021-12-10,True
141,4372,132,"28,8",CDU/CSU-Antrag: Verlängerung der epidemischen ...,2021-11-18,False


In [9]:
poll_ids = polls.data.id

all_votes = {}

for id in poll_ids:
    all_votes[id] = Dataset(name=f"poll_{id}", awde_endpoint="votes", awde_params={"poll": id})
    if all_votes[id].data is None:
        print(id, end=" ")
        all_votes[id].fetch()
        all_votes[id].save()

...............................................................................................................................................

In [ ]:
def transform_vote(data):
    df = data.copy()

    df["name"] = df.mandate.apply(lambda d: d["label"].split(" (Bundestag")[0] if d is not None else None)
    df["vote_id"] = df.id
    df["poll_id"] = df.poll.apply(lambda d: d["id"])
    df["fraction"] = df.fraction.apply(lambda d: d["label"].split(" (Bundestag")[0] if d is not None else None)


    df = df[[
        "poll_id",
        "vote_id",
        "name",
        "fraction",
        "vote",
        "reason_no_show",
        "reason_no_show_other",
    ]]

    return df

for k, v in all_votes.items():
    v.transform_data = transform_vote

allvotes = pd.concat([x.data for x in all_votes.values()])

In [62]:
allvotes

,poll_id,vote_id,name,fraction,vote,reason_no_show,reason_no_show_other
0,5593,575669,Uwe Witt,fraktionslos,no_show,None,None
1,5593,575668,Thomas Seitz,fraktionslos,yes,None,None
2,5593,575667,Stefan Seidler,fraktionslos,abstain,None,None
3,5593,575666,Johannes Huber,fraktionslos,yes,None,None
4,5593,575665,Matthias Helferich,fraktionslos,yes,None,None
...,...,...,...,...,...,...,...
731,4370,425231,Niels Annen,SPD,no_show,None,None
732,4370,425230,Dagmar Andres,SPD,yes,None,None
733,4370,425229,Reem Alabali-Radovan,SPD,yes,None,None
734,4370,425228,Adis Ahmetovic,SPD,yes,None,None
